In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


Generate random input data

In [20]:
y_fullyCon = torch.rand(5,129, 8)
y_conv = torch.rand(5, 1, 129,8) #5 input of dimension 128 by 8

print(y_conv.shape)

print(y_conv[1])


torch.Size([5, 1, 129, 8])
tensor([[[0.3449, 0.7370, 0.2471,  ..., 0.5792, 0.5624, 0.3328],
         [0.9019, 0.5994, 0.0437,  ..., 0.4856, 0.6751, 0.1265],
         [0.2022, 0.9487, 0.5585,  ..., 0.7653, 0.8653, 0.8247],
         ...,
         [0.2956, 0.5379, 0.4725,  ..., 0.1908, 0.1840, 0.6513],
         [0.8690, 0.9834, 0.8975,  ..., 0.5374, 0.8615, 0.3001],
         [0.7637, 0.7652, 0.7060,  ..., 0.2704, 0.6114, 0.8191]]])


Define the architecture for the fully connected model

In [21]:
class FullyConnectedBaseline(nn.Module):
    def __init__(self, n_features, n_segments):
        super(FullyConnectedBaseline, self).__init__()
        self.fc1 = nn.Linear(n_features*n_segments, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc2_bn = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(1024, n_features)

    def forward(self, x):
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.relu(self.fc2_bn(self.fc2(x)))
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Define the architecture for the convolutional model

In [22]:
class ConvolutionalBaseline(nn.Module):
    def __init__(self, n_features, n_segments):
        super(ConvolutionalBaseline, self).__init__()
        self.n_features = n_features
        self.n_segments = n_segments
        self.conv1 = nn.Conv2d(1, 18, kernel_size=(3,3), padding=(1,1))
        self.conv1_bn = nn.BatchNorm2d(18)
        self.conv2 = nn.Conv2d(18, 30, kernel_size=(3,3), padding=(1,1))
        self.conv2_bn = nn.BatchNorm2d(30)
        self.conv3 = nn.Conv2d(30, 40, kernel_size=(3,3), padding=(1,1))
        self.conv3_bn = nn.BatchNorm2d(40)
        self.fc1 = nn.Linear(40 * n_features * n_segments, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, n_features)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3(x)))
        print(x.shape)
        x = x.view(x.shape[0], -1)
        print(x.shape)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return x

In [23]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


model = ConvolutionalBaseline(129, 8)

model.apply(init_weights)

loss_list = []

#x = y.view(5, -1)
#print(x)
#print(x.shape)
result = model.forward(y_conv) 

print(y_conv.shape)
print(result.shape)


torch.Size([5, 40, 129, 8])
torch.Size([5, 41280])
torch.Size([5, 1, 129, 8])
torch.Size([5, 129])
